In [9]:
# Data manipulation
import pandas as pd
# Apply fast cleaning 
import numpy as np
# Geocode
from geopy.geocoders import Nominatim
# For timesleep
import time

In [10]:
df_sp = pd.read_csv(r"C:\Users\Alan\Desktop\Estabelecimentos_10_2024\extracted\Treated\empreendimentos_SP_SaoPaulo_202410301512.csv",
                    encoding='latin',
                    sep=';',
                    index_col=False,
                    dtype=str)

In [11]:
# Inspeção geral
df_sp.head(1)
df_sp.columns

Index(['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'matriz_filial',
       'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral',
       'motivo_situacao_cadastral', 'data_inicio_atividade', 'cnae_principal',
       'cnae_secundaria', 'tipo_logradouro', 'logradouro', 'numero',
       'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd1', 'tel1',
       'email', 'cnpj'],
      dtype='object')

In [ ]:
# Eliminar coluna adicional CNPJ
df_sp.drop(columns=['cnpj'], inplace=True)

# Coluna CNPJ Completo
df_sp['cnpj_completo'] = (
    df_sp['cnpj_basico'].str.zfill(8) + '/' + 
    df_sp['cnpj_ordem'].str.zfill(4) + '-' + 
    df_sp['cnpj_dv'].str.zfill(2)
)
# Coluna endereço completo CORRIGIR
df_sp['endereco_completo'] = (
    df_sp['logradouro'] + ', ' + 
    df_sp['numero'] + ', ' + 
    df_sp['bairro'] + ', ' + 
    df_sp['cep'] + ', ' + 
    df_sp['municipio'] + '-' + 
    df_sp['uf']
)

# Separando as CNAEs secundárias com max split 3
df_split = df_sp['cnae_secundaria'].str.split(',', n=2, expand=True)
df_split.columns = [f'cnae_secundaria{i+1}' for i in range(df_split.shape[1])]

# Unindo ao dataframe principal e trocando cnae_secundaria pelas do split
df_sp = df_sp.drop(columns=['cnae_secundaria']).join(df_split)

# Transformando colunas ano em datetime e eliminando as horas
df_sp['data_inicio_atividade'] = pd.to_datetime(df_sp['data_inicio_atividade'].astype(str), format='%Y%m%d', errors='coerce')
df_sp['data_inicio_atividade'] = df_sp['data_inicio_atividade'].dt.date

df_sp['data_situacao_cadastral'] = pd.to_datetime(df_sp['data_situacao_cadastral'].astype(str), format='%Y%m%d', errors='coerce')
df_sp['data_situacao_cadastral'] = df_sp['data_situacao_cadastral'].dt.date


In [13]:
# Padronização de None
df_sp = df_sp.where(pd.notna(df_sp), None)

In [14]:
df_sp.to_csv("Estabelecimentos_SaoPaulo_SP_20241031_nogeocode.csv", sep=';', encoding='latin1')

In [15]:
import os
print(os.getcwd())

C:\Users\Alan\AppData\Local\Programs\Microsoft VS Code
